# Introduction

put stuff here

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import requests
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}

In [2]:
# List of state abbreviations
states = ['AL', 'AK', 'AZ' ,'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN',
 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV',
 'NH', 'NJ', 'NM', 'NY', 'DC', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [3]:
list = []
x = 0
#df = pd.json_normalize(res.json())
for state in states:
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=MHLTH&$limit=4000&stateabbr={state}')
    list.append(res.json())
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=DEPRESSION&$limit=4000&stateabbr={state}')
    list.append(res.json())
    
    

In [163]:
df = pd.json_normalize(list[0])
df.set_index('locationid')
for i in range(1,len(list)):
    curr = pd.json_normalize(list[i])
    df = df.append(curr)


In [164]:
# Cleaning up DF a little bit

df = df.rename(columns={'year': 'Year', 'statedesc': 'State', 'locationname': 'City', 'measure': 'Symptom', 'data_value': 'Percent', 'stateabbr': 'State Abbr'})
del df['datasource']
del df['measureid']
del df['geolocation.type']
del df['geolocation.coordinates']
del df['data_value_footnote_symbol']
del df['data_value_footnote']
del df['low_confidence_limit']
del df['high_confidence_limit']
del df['categoryid']
del df['datavaluetypeid']

df['Affected Population'] = df['totalpopulation'].astype('int') * df['Percent'].astype('double') / 100

In [165]:
# Import Income Data Set
income_df = pd.read_csv("kaggle_income.csv", encoding='latin-1')
income_df['loc_fullname'] = income_df['City'] + ', ' + income_df['State_Name']

# delete unnecessary columns
del income_df['id']
del income_df['Type']
del income_df['County']
del income_df['Place']
del income_df['Primary']
del income_df['Lat']
del income_df['Lon']
del income_df['sum_w']
del income_df['Stdev']
del income_df['Area_Code']
del income_df['ALand']
del income_df['AWater']


# rename columns
income_df = income_df.rename(columns = {'Mean':'Mean Income','Median':'Median Income', 'Zip_Code': 'Zip Code'})

# combine duplicate cities
grouped = income_df.groupby(['loc_fullname'], as_index=False)
income_df = grouped.agg('mean')
income_df.head(5)

,loc_fullname,State_Code,Zip Code,Mean Income,Median Income
0,"Abbeville, Alabama",1.0,36310.0,40518.000000,25216.000000
1,"Abbeville, Louisiana",22.0,70510.0,30647.000000,25628.333333
2,"Abbeville, South Carolina",45.0,29620.0,45585.666667,32245.000000
3,"Abbotsford, Wisconsin",55.0,54405.0,58254.000000,44919.000000
4,"Aberdeen, Maryland",24.0,21001.0,96480.000000,80370.000000


In [166]:
# import us cities dataset
cities_df = pd.read_csv('uscities.csv')
del cities_df['city_ascii']
del cities_df['lat']
del cities_df['lng']
del cities_df['source']
del cities_df['military']
del cities_df['incorporated']
del cities_df['timezone']
del cities_df['ranking']
del cities_df['id']

cities_df = cities_df.rename(columns={'population': 'Total Population', 'city': 'City', 'state_id': 'State Abbr',
                                     'state_name': 'State', 'county_name': 'County', 'density': 'Density'})
cities_df['loc_fullname'] = cities_df['City'] + ', ' + cities_df['State']
cities_df['County'] = cities_df['County'] + ' County'

cities_df.head(5)

,City,State Abbr,State,county_fips,County,Total Population,Density,zips,loc_fullname
0,New York,NY,New York,36061,New York County,18713220,10715,11229 11226 11225 11224 11222 11221 11220 1138...,"New York, New York"
1,Los Angeles,CA,California,6037,Los Angeles County,12750807,3276,90291 90293 90292 91316 91311 90037 90031 9000...,"Los Angeles, California"
2,Chicago,IL,Illinois,17031,Cook County,8604203,4574,60018 60649 60641 60640 60643 60642 60645 6064...,"Chicago, Illinois"
3,Miami,FL,Florida,12086,Miami-Dade County,6445545,5019,33129 33125 33126 33127 33128 33149 33144 3314...,"Miami, Florida"
4,Dallas,TX,Texas,48113,Dallas County,5743938,1526,75287 75098 75233 75254 75251 75252 75253 7503...,"Dallas, Texas"


In [167]:
# lets also do temperature by county, import from this dataset

temperature = pd.read_csv('temp_by_county.csv')

temperature = temperature.rename(columns={'Location': 'County', '1901-2000 Mean': 'Annual Avg Temp F'})

# this data is formatted pretty weird
# lets add a state column so we can more reasily merge it back onto our main df
temperature['State Abbr'] = temperature['Location ID'].apply(lambda x: x[:2])
temperature.head(5)

,Location ID,County,Value,Rank,Anomaly (1901-2000 base period),Annual Avg Temp F,State Abbr
0,AL-001,Autauga County,53.0,48,-1.3,54.3,AL
1,AL-003,Baldwin County,57.6,61,-0.7,58.3,AL
2,AL-005,Barbour County,54.6,47,-1.2,55.8,AL
3,AL-007,Bibb County,51.2,44,-1.6,52.8,AL
4,AL-009,Blount County,50.3,61,-0.3,50.6,AL


In [168]:
df.head(1)

,Year,State Abbr,State,City,category,Symptom,data_value_unit,data_value_type,Percent,totalpopulation,locationid,short_question_text,Affected Population
0,2019,AL,Alabama,Childersburg,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,21.7,5175,0114464,Mental Health,1122.975


In [169]:
# Create a column that holds: {City}, {State}
# We will merge using this as the key. Then we will delete this (because we already have columns for state and cities)

df['loc_fullname'] = df['City'] + ', ' + df['State']
df = df.merge(income_df[['loc_fullname', 'Mean Income', 'Median Income', 'Zip Code']], on='loc_fullname')
df = df.merge(cities_df[['loc_fullname', 'Total Population', 'County', 'Density']], on='loc_fullname')
df = df.merge(temperature, on=['State Abbr', 'County'])
del df['loc_fullname']

In [170]:
df['Mental Health Age-Adjusted Percent'] = (df.loc[((df['short_question_text'] == 'Mental Health') & (df['data_value_type'] == 'Age-adjusted prevalence'))]['Percent']).astype('double')
df['Mental Health Crude Percent'] = (df.loc[((df['short_question_text'] == 'Mental Health') & (df['data_value_type'] == 'Crude prevalence'))]['Percent']).astype('double')
df['Depression Age-Adjusted Percent'] = (df.loc[((df['short_question_text'] == 'Depression') & (df['data_value_type'] == 'Age-adjusted prevalence'))]['Percent']).astype('double')
df['Depression Crude Percent'] = (df.loc[((df['short_question_text'] == 'Depression') & (df['data_value_type'] == 'Crude prevalence'))]['Percent']).astype('double')


In [171]:
df.head(5)

,Year,State Abbr,State,City,category,Symptom,data_value_unit,data_value_type,Percent,totalpopulation,...,Density,Location ID,Value,Rank,Anomaly (1901-2000 base period),Annual Avg Temp F,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
0,2019,AL,Alabama,Childersburg,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,21.7,5175,...,151,AL-121,50.8,50,-1.0,51.8,21.7,NaN,NaN,NaN
1,2019,AL,Alabama,Childersburg,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,21.2,5175,...,151,AL-121,50.8,50,-1.0,51.8,NaN,21.2,NaN,NaN
2,2019,AL,Alabama,Childersburg,Health Outcomes,Depression among adults aged >=18 years,%,Age-adjusted prevalence,27.5,5175,...,151,AL-121,50.8,50,-1.0,51.8,NaN,NaN,27.5,NaN
3,2019,AL,Alabama,Childersburg,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,27.3,5175,...,151,AL-121,50.8,50,-1.0,51.8,NaN,NaN,NaN,27.3
4,2019,AL,Alabama,Sylacauga,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,19.7,12831,...,229,AL-121,50.8,50,-1.0,51.8,NaN,19.7,NaN,NaN


In [172]:
df.head(1)

,Year,State Abbr,State,City,category,Symptom,data_value_unit,data_value_type,Percent,totalpopulation,...,Density,Location ID,Value,Rank,Anomaly (1901-2000 base period),Annual Avg Temp F,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
0,2019,AL,Alabama,Childersburg,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,21.7,5175,...,151,AL-121,50.8,50,-1.0,51.8,21.7,NaN,NaN,NaN


In [173]:
del df['Symptom']
del df['data_value_unit']
del df['data_value_type']
del df['Percent']
del df['short_question_text']
del df['Location ID']
del df['Value']
del df['Rank']
del df['Anomaly (1901-2000 base period)']
del df['category']
del df['totalpopulation']
df['Zip Code'] = df['Zip Code'].astype('int')

In [174]:
# Aggregate rows to have one instance of each and information on mental health and depression rates
grouped = df.groupby(by=['Year','State', 'State Abbr','City','locationid','Mean Income','Median Income','Zip Code'
                         ,'Total Population', 'County', 'Annual Avg Temp F'], as_index=False, dropna=False)
df = grouped.agg('sum')

df.set_index('locationid',inplace = True)

# Reordering columns
df = df[['Year', 'State Abbr', 'State', 'City', 'County', 'Mean Income', 'Median Income', 'Annual Avg Temp F'
         ,'Total Population', 'Density', 'Affected Population', 'Mental Health Age-Adjusted Percent', 'Mental Health Crude Percent', 'Depression Age-Adjusted Percent',
        'Depression Crude Percent']]

In [175]:
df.head(5)

,Year,State Abbr,State,City,County,Mean Income,Median Income,Annual Avg Temp F,Total Population,Density,Affected Population,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
locationid,,,,,,,,,,,,,,,
0100124,2019,AL,Alabama,Abbeville,Henry County,40518.000000,25216.000000,56.8,2560,252,2335.872,20.2,18.3,24.8,23.6
0100460,2019,AL,Alabama,Adamsville,Jefferson County,53383.000000,47460.000000,51.7,4281,260,3789.436,18.8,18.1,23.6,23.3
0100820,2019,AL,Alabama,Alabaster,Shelby County,81118.333333,149711.333333,52.1,33487,2048,24466.260,15.4,15.6,23.3,23.7
0100988,2019,AL,Alabama,Albertville,Marshall County,47199.500000,160358.500000,50.5,37775,1252,20303.028,19.4,19.5,28.2,28.3
0101132,2019,AL,Alabama,Alexander City,Tallapoosa County,40286.000000,31306.000000,52.8,14317,524,13429.884,19.5,18.6,26.1,25.5


In [176]:
df.loc[df['Mean Income'].astype('int') < 1000]
# As per our data source, 0 income means that there are no records of income for these cities. 
# We can either make these values NAN or completely get rid of these rows. 
# We decided to just delete these rows as none of them are extremely 'important' cities. 

,Year,State Abbr,State,City,County,Mean Income,Median Income,Annual Avg Temp F,Total Population,Density,Affected Population,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
locationid,,,,,,,,,,,,,,,
0179008,2019,AL,Alabama,Vincent,Shelby County,0.0,0.0,52.1,2213,164,1940.424,20.6,19.8,26.6,26.2
0418160,2019,AZ,Arizona,Dateland,Yuma County,0.0,0.0,60.7,434,28,256.256,13.8,13.3,17.3,17.2
0603302,2019,CA,California,Avenal,Kings County,0.0,0.0,53.2,13496,1068,10682.945,16.9,18.2,16.6,17.2
0823300,2019,CO,Colorado,Edwards,Eagle County,0.0,0.0,27.8,9350,540,6508.644,12.9,13.5,18.1,18.9
1048330,2019,DE,Delaware,Milton,Sussex County,0.0,0.0,46.5,3012,2600,1666.672,13.9,13.4,18.8,18.6
1211225,2019,FL,Florida,Cedar Key,Levy County,0.0,0.0,62.1,720,1100,558.446,19.3,15.8,23.2,20.8
1251075,2019,FL,Florida,Ocoee,Orange County,0.0,0.0,66.1,48263,4868,26082.056,16.0,16.3,20.1,20.4
1723737,2019,IL,Illinois,El Paso,Woodford County,0.0,0.0,38.7,2730,1964,2124.360,15.3,14.7,23.1,22.5
1768640,2019,IL,Illinois,Seneca,LaSalle County,0.0,0.0,38.2,2251,532,1801.960,15.5,15.3,22.6,22.6


In [177]:
df = df.drop(df[df['Mean Income'].astype('int') < 1000].index)
df

,Year,State Abbr,State,City,County,Mean Income,Median Income,Annual Avg Temp F,Total Population,Density,Affected Population,Mental Health Age-Adjusted Percent,Mental Health Crude Percent,Depression Age-Adjusted Percent,Depression Crude Percent
locationid,,,,,,,,,,,,,,,
0100124,2019,AL,Alabama,Abbeville,Henry County,40518.000000,25216.000000,56.8,2560,252,2335.872,20.2,18.3,24.8,23.6
0100460,2019,AL,Alabama,Adamsville,Jefferson County,53383.000000,47460.000000,51.7,4281,260,3789.436,18.8,18.1,23.6,23.3
0100820,2019,AL,Alabama,Alabaster,Shelby County,81118.333333,149711.333333,52.1,33487,2048,24466.260,15.4,15.6,23.3,23.7
0100988,2019,AL,Alabama,Albertville,Marshall County,47199.500000,160358.500000,50.5,37775,1252,20303.028,19.4,19.5,28.2,28.3
0101132,2019,AL,Alabama,Alexander City,Tallapoosa County,40286.000000,31306.000000,52.8,14317,524,13429.884,19.5,18.6,26.1,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5677530,2019,WY,Wyoming,Torrington,Goshen County,47383.000000,171237.500000,34.0,7289,2208,4173.642,13.8,13.2,18.9,18.3
5679125,2019,WY,Wyoming,Upton,Weston County,69215.000000,300000.000000,31.1,1056,548,717.200,14.4,13.8,18.8,18.2
5683040,2019,WY,Wyoming,Wheatland,Platte County,87954.000000,164106.000000,33.7,3462,1304,2357.550,14.0,12.8,19.6,18.6


In [178]:
# That was all collection and processing. Now exploratory analysis and Visualization

# Some things we could possibly explore in analysis
    # most depressed cities
    # avg depression/mental health rate by city
    # state with worst depression, state with worst mental health
    # income vs depression rates
    # population density vs depression/mental health rates
    # temp vs depression/mental health rates
    # for every city, mental health rates vs depression rates, similar?
    # maybe try using folium, geopandas, basemap. Couldn't get any to work atm
        # talk about regions that seem to have higher rates of mental illness/depression
        # if we dont do this we could also keep longitude/latitude data, plot a distribution